In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("sample.csv")
df

,Unnamed: 0,reviewText,overall
0,0,Just another flavor of Kit Kat but the taste i...,0.0
1,1,I bought this on impulse and it comes from Jap...,0.0
2,2,Really good. Great gift for any fan of green t...,0.0
3,3,"I had never had it before, was curious to see ...",1.0
4,4,I've been looking forward to trying these afte...,0.0
...,...,...,...
994,994,I will never substitute this Sweet Leaf brand ...,1.0
995,995,I've made my own extract several times but i'm...,1.0
996,996,I switched from Splenda to this stuff about a ...,1.0
997,997,Love this stevia.It is the only brand I use.Wi...,1.0


In [6]:
df.drop("Unnamed: 0",   axis='columns', inplace=True)
df

,reviewText,overall
0,Just another flavor of Kit Kat but the taste i...,0.0
1,I bought this on impulse and it comes from Jap...,0.0
2,Really good. Great gift for any fan of green t...,0.0
3,"I had never had it before, was curious to see ...",1.0
4,I've been looking forward to trying these afte...,0.0
...,...,...
994,I will never substitute this Sweet Leaf brand ...,1.0
995,I've made my own extract several times but i'm...,1.0
996,I switched from Splenda to this stuff about a ...,1.0
997,Love this stevia.It is the only brand I use.Wi...,1.0


In [9]:
## Drop Nan values:
df = df.dropna()

In [12]:
#Renaming columns:
df.columns = ["reviewText", "label"]
df.head()

,reviewText,label
0,Just another flavor of Kit Kat but the taste i...,0.0
1,I bought this on impulse and it comes from Jap...,0.0
2,Really good. Great gift for any fan of green t...,0.0
3,"I had never had it before, was curious to see ...",1.0
4,I've been looking forward to trying these afte...,0.0


In [13]:
##Get the independent features
X = df.drop("label", axis=1)

In [14]:
## Get the Dependent features
y=df["label"]

In [17]:
y.value_counts()

1.0    700
0.0    299
Name: label, dtype: int64

In [15]:
X.shape

(999, 1)

In [16]:
y.shape

(999,)

In [18]:
import tensorflow as tf

In [19]:
tf.__version__

'2.1.0'

In [20]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [21]:
### Vocabulary size
voc_size=5000

## Data Preprocessing

In [23]:
#Creating a copy of X into the variable messages
messages = X.copy()

In [26]:
messages['reviewText'][0]

'Just another flavor of Kit Kat but the taste is unique and a bit different.  The only thing that is bothersome is the price.  I thought it was a bit expensive....'

In [24]:
#Reset index as we drop Nan values from df
messages.reset_index(inplace=True)

In [28]:
import nltk
import re
from nltk.corpus import stopwords

In [31]:
#In order to remove words that are not meanningful (e.g. the, a, then, often...), we need to download those words.
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Babette\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
##Data Preprocessing/Cleaning
from nltk.stem.porter import PorterStemmer
#Initialise PorterStemmer for Stemming
ps = PorterStemmer()
#Create an empty list named corpus that will contain our cleaned sentences and words
corpus = []
#Create a loop to clean all the text in messages:
for i in range(0, len(messages)):
    #print index
    print(i)
    #use re (regular expressions) to substitute all characters except [a-zA-Z] by blank in message 'reviewText'
    review = re.sub('[^a-zA-Z]', ' ', messages['reviewText'][i])
    #convert all the characters as lower case
    review = review.lower()
    #split all the words in each sentence to be able to later remove the stopwords
    review = review.split()
    
    #create a loop in review: for each word in review, keep only words that are not stopwords list and apply 'Stemming'
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #join words with a space to build the review
    review = ' '.join(review)
    #append the review into the corpus
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [37]:
corpus

['anoth flavor kit kat tast uniqu bit differ thing bothersom price thought bit expens',
 'bought impuls come japan amus famili weird stamp mark packag fun said would take month arriv take long hope interest tast famili tast bit less flavor weaker standard milk chocol kit kat green tea flavor subtl sugar tast sweet wafer crispi good tast bit anem us happi bought need buy',
 'realli good great gift fan green tea expens purchas candi across sea',
 'never curiou see like smooth great subtl good flavor order plan make routin',
 'look forward tri hear popular japan among kit kat fan well recommend order warm weather melt becom smushi order mine right summer began bit mushi let solidifi room temp afterward tri tast fine expect stronger green tea matcha flavor actual quit subtl outer coat creami overli sugari like overal say insan good definit yummi treat',
 'kit kat good look strong green tea flavor disappoint green tea flavor subtl',
 'found mitsuwa marketplac illinoi actual expect tast bett

In [36]:
#Checking the new sentences in the corpus
for i in range(0,5):
    print(corpus[i])

anoth flavor kit kat tast uniqu bit differ thing bothersom price thought bit expens
bought impuls come japan amus famili weird stamp mark packag fun said would take month arriv take long hope interest tast famili tast bit less flavor weaker standard milk chocol kit kat green tea flavor subtl sugar tast sweet wafer crispi good tast bit anem us happi bought need buy
realli good great gift fan green tea expens purchas candi across sea
never curiou see like smooth great subtl good flavor order plan make routin
look forward tri hear popular japan among kit kat fan well recommend order warm weather melt becom smushi order mine right summer began bit mushi let solidifi room temp afterward tri tast fine expect stronger green tea matcha flavor actual quit subtl outer coat creami overli sugari like overal say insan good definit yummi treat


## One-Hot representation of words/sentences

In [38]:
#Apply One_hot representation for each word in the corpus based on the voc_size - each word is allocated a number within the sentence.
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[960,
  719,
  119,
  1656,
  3957,
  4077,
  4258,
  2342,
  4466,
  2594,
  1955,
  2707,
  4258,
  3278],
 [421,
  3531,
  2878,
  4634,
  2008,
  3925,
  141,
  2727,
  1398,
  2890,
  3056,
  65,
  1748,
  3724,
  802,
  374,
  3724,
  1834,
  2473,
  632,
  3957,
  3925,
  3957,
  4258,
  942,
  719,
  4237,
  2082,
  288,
  2023,
  119,
  1656,
  3132,
  4460,
  719,
  178,
  586,
  3957,
  2237,
  4263,
  1563,
  4129,
  3957,
  4258,
  3547,
  2617,
  2399,
  421,
  2724,
  673],
 [3994, 4129, 3097, 4345, 3005, 3132, 4460, 3278, 2411, 2886, 4885, 601],
 [3464, 2911, 466, 2709, 1624, 3097, 178, 4129, 719, 666, 1960, 2879, 239],
 [717,
  4038,
  2085,
  2172,
  606,
  4634,
  1443,
  119,
  1656,
  3005,
  4209,
  3733,
  666,
  1054,
  1418,
  2484,
  2349,
  4035,
  666,
  3816,
  2279,
  1390,
  1029,
  4258,
  523,
  3228,
  537,
  3914,
  1128,
  3783,
  2085,
  3957,
  2601,
  1625,
  4322,
  3132,
  4460,
  4143,
  719,
  2100,
  1098,
  178,
  2925,
  1379,
  3939,
  10

In [72]:
#Checking the maximum length of all sentences.
number_words=[]
for i in range(0,5):
    number_words.append(len(onehot_repr[i]))
number_words

[14, 50, 12, 13, 55]

In [70]:
A = [-1, 3, 7, 99, 0]
max(A)


TypeError: 'list' object is not callable

## Word Embedding

In [ ]:
#Use pad sequencing to ensure all sentences are the same length - same number of words
